In [ ]:
!pip install ultralytics pytesseract opencv-python scikit-image
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install ultralytics pytesseract opencv-python scikit-image

import torch
from ultralytics import YOLO
import pytesseract
import cv2
import os
from skimage import color, io
import numpy as np

model = YOLO("yolo11s.pt")

save_dir = "detection_results"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)


def process_video(video_path):
    results = model(video_path, save=True, project=save_dir)


    for result in results:
        if result.boxes is not None:
            boxes = result.boxes
            detected_objects = []

            for box in boxes:
                try:

                    cls_id = int(box.cls)
                    class_name = result.names[cls_id]

                    detected_objects.append(class_name)

                    if class_name in ['apple', 'banana', 'tomato']:
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        cropped_img = result.orig_img[y1:y2, x1:x2]

                        if cropped_img.size > 0:
                            freshness_status = assess_real_freshness(cropped_img)
                            print(f"Detected {class_name} with real-life freshness: {freshness_status}")
                        else:
                            print("Empty cropped image for freshness detection.")

        print(f"Results saved in: {result.save_dir}")

video_path = "/content/1.mp4"
if os.path.exists(video_path):
    process_video(video_path)
else:
    print(f"Video file not found: {video_path}")
